Instalace potřebných modulů: ipython, numpy, pandas, sklearn

In [1]:
!pip install ipython numpy pandas scikit-learn

Načtení potřebných modulů

In [2]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from IPython.display import display

Načtení a zpracování dat

In [3]:
# Načte data ze soubor
try:
    # Nejdříve zkusí excel soubor
    DATA = pd.read_excel("data.xlsx")
except:
    # Poté zkusí csv soubor
    # Pokud hlásí chybu, je možné nastavit vlastní:
    # > delimiter ... oddělovač dat (čárka, středník)
    # > decimal ... oddělovač desetinných míst (čírka, tečka)
    DATA = pd.read_csv("data.csv", delimiter=",", decimal=",")
display(DATA)
# Odstraní první sloupeček
data = DATA.drop(DATA.columns[0], axis=1, inplace=False)
# Získá indexy řídků, kde chybí číslo v posledním sloupečku
indexy_pro_predikci = data.index[data[data.columns[-1]].isnull().values].tolist()
# Získá řádky, které se mohou použít pro regresi
data_pro_predikci = data.loc[indexy_pro_predikci].drop(data.columns[-1], axis=1, inplace=False)
data = data.loc[data.index.difference(indexy_pro_predikci)]
# Převedení dat do číselných polí
data_numbers = data.to_numpy()
vstup_pro_regresi = data_numbers[:,:-1]
vystup_pro_regresi = data_numbers[:,-1]
vstup_pro_predikci = data_pro_predikci.to_numpy()

,Projekt,m2,vzdálenost,počet modulů,počet vývozních dní,průměrná váha modulu,průměrná výška modulu,průměrná šířka modulu,průměrná délka modulu,cena kalkulovaná Euro,cena skutečná
0,Dueren,1500,600,43,3,20,3,3.5,14,150000,200000.0
1,Pirna II,1200,250,22,2,18,3,4.0,16,170000,155000.0
2,Ronnenberg,800,550,11,1,25,3,4.0,16,170000,190000.0
3,Predikce 1,820,520,8,2,25,4,4.5,18,160000,NaN


Vytvoření predikčního modelu pomocí lineární regrese

https://scikit-learn.org/stable/tutorial/statistical_inference/supervised_learning.html#linear-model-from-regression-to-sparsity

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [4]:
regrese = linear_model.LinearRegression()
regrese.fit(vstup_pro_regresi, vystup_pro_regresi)

# Výpis parametrů modelu
print("Lineární koeficienty:", [*regrese.coef_, regrese.intercept_])
print("Počet vstupních sloupců:", (regrese.rank_, regrese.n_features_in_))
print(
    "Průměrná chyba R2:",
    np.mean((regrese.predict(vstup_pro_regresi) - vystup_pro_regresi)**2),
    "(menší = lepší)"
)
print(
    "Skóre modelu:",
    regrese.score(vstup_pro_regresi, vystup_pro_regresi),
    "(lze predikovat? 0 = NE, 1 = ANO)"
)

Lineární koeficienty: [-55.929259470886365, 42.01465674067706, -1.5365531924001918, -0.1397937086174281, 0.9796158024743624, 0.0, -5.888011957586796e-05, -0.00023552047830347183, -2.3552047830346226, 612012.7149728338]
Počet vstupních sloupců: (2, 9)
Průměrná chyba R2: 4.291633599421788e-20 (menší = lepší)
Skóre modelu: 1.0 (lze predikovat? 0 = NE, 1 = ANO)


Predikce

In [5]:
vysledek_predikce = regrese.predict(vstup_pro_predikci)

Uložení predikovaných dat

In [6]:
DATA.loc[indexy_pro_predikci, DATA.columns[-1]] = vysledek_predikce
DATA.to_excel("data_new.xlsx")